In [ ]:
# Describtion: 

In [1]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import torch.nn as nn


import itertools
import os
from utils import *


os.environ["DGLBACKEND"] = "pytorch"

import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import torch.nn.utils.prune as prune
import statistics as stat
from tabulate import tabulate


In [2]:
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.path.abspath("../"))

import torch
import torch_pruning as tp

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')


In [4]:
def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for param in model.parameters():
        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


def get_model_size(model: nn.Module, data_width=32, count_nonzero_only=False) -> int:
    """
    calculate the model size in bits
    :param data_width: #bits per element
    :param count_nonzero_only: only count nonzero weights
    """
    return get_num_parameters(model, count_nonzero_only) * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

from sklearn.model_selection import StratifiedKFold
folds=5

def k_fold(dataset, folds):
    skf = StratifiedKFold(folds, shuffle=True, random_state=12345)

    test_indices, train_indices = [], []
    X = dataset.data.x  # or whatever your data is
    y = dataset.data.y
    #print('dataset.data.x :', dataset.data.x.shape)
    #print('dataset.data.y :', dataset.data.y.shape)
    for _, idx in skf.split(torch.zeros(len(X)), y):
        test_indices.append(torch.from_numpy(idx))

    val_indices = [test_indices[i % len(test_indices)] for i in range(folds)]


    for i in range(folds):
        train_mask = torch.ones(len(X), dtype=torch.bool)
        train_mask[test_indices[i]] = 0
        train_mask[val_indices[i]] = 0
        train_indices.append(train_mask.nonzero().view(-1))
        
    return train_indices, test_indices, val_indices  



# Convert edge_index to adjacency matrix
adj_matrix = torch.zeros((dataset.data.num_nodes, dataset.data.num_nodes))
adj_matrix[dataset.data.edge_index[0], dataset.data.edge_index[1]] = 1

def build_edge_index(data_idx):
        data_adj_matrix = adj_matrix[data_idx.tolist()][:, data_idx.tolist()]
        # Optionally, convert adjacency matrices back to edge_index tensors
        data_edge_index = torch.nonzero(data_adj_matrix, as_tuple=False)
        row1 = data_edge_index[:, 1]  # Second column
        row2 = data_edge_index[:, 0]  # First column
        # Combine rows to create the desired output tensor
        data_edge_index = torch.stack([row1, row2], dim=0)
        return data_edge_index

In [5]:
def train(model, optimizer, data):
    model.train()


    optimizer.zero_grad()
    data = data.to(device)
    out = model(data)
    loss = F.cross_entropy(out, data.y.view(-1))   
    loss.backward()   
    optimizer.step()
    return loss


def eval_loss(model, data):
    model.eval()
    loss = 0
    data = data.to(device)
    print
    with torch.no_grad():
        out = model(data)
        
    loss = F.cross_entropy(out, data.y.view(-1), reduction="sum").item()
    print('eval_loss:', loss)
    
    return loss / data.num_nodes


def eval_acc(model, data):
    model.eval()
    #correct = 0
    #for data in loader:
    data = data.to(device)
    with torch.no_grad():
        pred = model(data).max(1)[1]
 
    
    correct = pred.eq(data.y).sum().item()
    return correct / data.num_nodes


In [6]:
#Main model

import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self,dataset):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, 16)  # Assuming you want to keep the intermediate features consistent
        self.linear1= torch.nn.Linear(16, 16 ) # Linear layer for final classification
        self.linear2= torch.nn.Linear(16, dataset.num_classes)  # Linear layer for final classification

    def forward(self, data):
        x, edge_index = data.data, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        # Apply linear transformation
        x = self.linear1(x)
        x = self.linear2(x)
                          
        
    
        return x


In [7]:
from tqdm import tqdm
accu = []
accu_max = []
max_acc = 0.85

for fold, (train_idx, test_idx, val_idx) in enumerate(zip(*k_fold(dataset,5))):
        print_max_acc=0
        
        train_dataset = dataset.x[train_idx.tolist()]
        train_dataset.y=dataset.y[train_idx.tolist()]
        train_dataset.edge_index =build_edge_index(train_idx)
        train_dataset.num_features=dataset.num_features
        train_dataset.num_classes=dataset.num_classes
        train_dataset.num_nodes=len(train_dataset)
        
        test_dataset = dataset.x[test_idx.tolist()]
        test_dataset.y=dataset.y[test_idx.tolist()]
        test_dataset.edge_index = build_edge_index(test_idx) 
        test_dataset.num_nodes=len(test_dataset)
        
        val_dataset = dataset.x[val_idx.tolist()]
        val_dataset.y=dataset.y[val_idx.tolist()]
        val_dataset.edge_index =build_edge_index(val_idx)
        val_dataset.num_nodes=len(val_dataset)

        
    
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = GCN(dataset=train_dataset).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
        
        
        for epoch in range(200):
                        t = tqdm(epoch)
                        train_loss=0
                        train_loss = train(model,optimizer,train_dataset)

                        start = time.process_time()
                        val_loss = eval_loss(model,val_dataset)
                        end = time.process_time()
                        acc = eval_acc(model,test_dataset)
                        t.set_postfix(
                                    {
                                        "Train_Loss": "{:05.3f}".format(train_loss),
                                        "Val_Loss": "{:05.3f}".format(val_loss),
                                        "Acc": "{:05.3f}".format(acc),
                                        "Epoch":"{:05.1f}".format(epoch),
                                        "Fold":"{:05.1f}".format(fold),
                                    }
                                )
                        accu.append(acc)
                        if(acc>max_acc):
                            path = path2check+'/'+'GCN'+'_'+str(hidden_units)+'_on_'+'Cora'+'_'+str(bit)+'bit-'+str(max_epoch)+'.pth.tar'
                            max_acc = acc
                            torch.save({'state_dict': model.state_dict(), 'best_accu': acc,}, path)
                        if(acc>print_max_acc):
                            print_max_acc = acc
                       
                       
                        

0it [00:00, ?it/s, Train_Loss=1.993, Val_Loss=1.947, Acc=0.085, Epoch=000.0, Fold=000.0]

eval_loss: 1055.53955078125



0it [00:00, ?it/s, Train_Loss=1.993, Val_Loss=1.947, Acc=0.085, Epoch=000.0, Fold=000.0]

0it [00:00, ?it/s, Train_Loss=1.945, Val_Loss=1.898, Acc=0.297, Epoch=001.0, Fold=000.0]

eval_loss: 1028.9830322265625


0it [00:00, ?it/s, Train_Loss=1.945, Val_Loss=1.898, Acc=0.297, Epoch=001.0, Fold=000.0]
0it [00:00, ?it/s, Train_Loss=1.894, Val_Loss=1.842, Acc=0.304, Epoch=002.0, Fold=000.0]

...

eval_loss: 1056.7254638671875


0it [00:00, ?it/s, Train_Loss=0.125, Val_Loss=1.953, Acc=0.725, Epoch=197.0, Fold=004.0]
0it [00:00, ?it/s, Train_Loss=0.131, Val_Loss=1.905, Acc=0.721, Epoch=198.0, Fold=004.0]

eval_loss: 1030.3521728515625



0it [00:00, ?it/s, Train_Loss=0.131, Val_Loss=1.905, Acc=0.721, Epoch=198.0, Fold=004.0]

0it [00:00, ?it/s, Train_Loss=0.123, Val_Loss=1.884, Acc=0.713, Epoch=199.0, Fold=004.0]

eval_loss: 1019.3031616210938


In [ ]:
# pdb.set_trace()
max_epoch=200
accu = torch.tensor(accu)
accu = accu.view(folds,max_epoch)
_, argmax = accu.max(dim=1)
#accu = accu[torch.arange(args.folds, dtype=torch.long), argmax]
acc_mean = accu.mean().item()
acc_std = accu.std().item()
desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
print(desc)


In [14]:

# report test msg

t0=time.time()
base_model_accuracy=eval_acc(model,test_dataset)
t1=time.time()
t_base_model=t1 - t0
###
base_model_size = get_model_size(model,count_nonzero_only=True)
num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)
###   
print(f"dense model has accuracy on test set={base_model_accuracy:.2f}%")
print(f"dense model has size={base_model_size/MiB:.2f} MiB")
print(f"The time inference of base model is ={t_base_model}") 
print(f"The number of parametrs of base model is:{num_parm_base_model}")  

dense model has accuracy on test set=0.72%
dense model has size=0.09 MiB
The time inference of base model is =0.008979082107543945
The number of parametrs of base model is:23607


## Manual Measurement

In [15]:

Eva_final=dict()

Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]


In [16]:
for i in range(5):
    accu = []
    accu_max = []
    max_acc = 0.8
    print(f'The iteration is :{i+1} ')
    Eva=dict()
    
    for fold, (train_idx, test_idx, val_idx) in enumerate(zip(*k_fold(dataset,5))):
            print_max_acc=0

            train_dataset = dataset.x[train_idx.tolist()]
            train_dataset.y=dataset.y[train_idx.tolist()]
            train_dataset.edge_index =build_edge_index(train_idx)
            train_dataset.num_features=dataset.num_features
            train_dataset.num_classes=dataset.num_classes
            train_dataset.num_nodes=len(train_dataset)

            test_dataset = dataset.x[test_idx.tolist()]
            test_dataset.y=dataset.y[test_idx.tolist()]
            test_dataset.edge_index = build_edge_index(test_idx) 
            test_dataset.num_nodes=len(test_dataset)

            val_dataset = dataset.x[val_idx.tolist()]
            val_dataset.y=dataset.y[val_idx.tolist()]
            val_dataset.edge_index =build_edge_index(val_idx)
            val_dataset.num_nodes=len(val_dataset)



            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = GCN(dataset=train_dataset).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


            for epoch in range(200):
                            t = tqdm(epoch)
                            train_loss=0
                            train_loss = train(model,optimizer,train_dataset)

                            start = time.process_time()
                            val_loss = eval_loss(model,val_dataset)
                            end = time.process_time()
                            acc = eval_acc(model,test_dataset)
                            t.set_postfix(
                                        {
                                            "Train_Loss": "{:05.3f}".format(train_loss),
                                            "Val_Loss": "{:05.3f}".format(val_loss),
                                            "Acc": "{:05.3f}".format(acc),
                                            "Epoch":"{:05.1f}".format(epoch),
                                            "Fold":"{:05.1f}".format(fold),
                                        }
                                    )
                            accu.append(acc)
                            if(acc>max_acc):
                                path = path2check+'/'+'GCN'+'_'+str(hidden_units)+'_on_'+'Cora'+'_'+str(bit)+'bit-'+str(max_epoch)+'.pth.tar'
                                max_acc = acc
                                torch.save({'state_dict': model.state_dict(), 'best_accu': acc,}, path)
                            if(acc>print_max_acc):
                                print_max_acc = acc

    # report test msg

    t0=time.time()
    base_model_accuracy=eval_acc(model,test_dataset)
    t1=time.time()
    t_base_model=t1 - t0
    ###
    base_model_size = get_model_size(model,count_nonzero_only=True)
    num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)
    ###   
    print(f"dense model has accuracy on test set={base_model_accuracy:.2f}%")
    print(f"dense model has size={base_model_size/MiB:.2f} MiB")
    print(f"The time inference of base model is ={t_base_model}") 
    print(f"The number of parametrs of base model is:{num_parm_base_model}")    
    
    #Update my Eva dictionary
    Eva.update({'base model accuracy': base_model_accuracy,
                'time inference of base model': t_base_model,
                'number parmameters of base model': num_parm_base_model,
                'size of base model': base_model_size})
    
    
  


    Base_model_accuracy.append(Eva['base model accuracy'])
    T_base_model.append(Eva['time inference of base model'])
    Num_parm_base_model.append(int(Eva['number parmameters of base model']))
    Base_model_size.append(int(Eva['size of base model']))




                                

The iteration is :1 


0it [00:18, ?it/s, Train_Loss=0.117, Val_Loss=1.976, Acc=0.723, Epoch=199.0, Fold=004.0]

eval_loss: 1048.1553955078125



0it [00:00, ?it/s, Train_Loss=1.975, Val_Loss=1.934, Acc=0.148, Epoch=000.0, Fold=000.0]
0it [00:00, ?it/s, Train_Loss=1.975, Val_Loss=1.934, Acc=0.148, Epoch=000.0, Fold=000.0]

0it [00:00, ?it/s, Train_Loss=1.931, Val_Loss=1.879, Acc=0.290, Epoch=001.0, Fold=000.0]

eval_loss: 1018.4520263671875


...

eval_loss: 967.01513671875



0it [00:00, ?it/s, Train_Loss=0.112, Val_Loss=1.787, Acc=0.723, Epoch=198.0, Fold=004.0]
0it [00:00, ?it/s, Train_Loss=0.112, Val_Loss=1.787, Acc=0.723, Epoch=198.0, Fold=004.0]



eval_loss: 969.9140014648438


0it [00:00, ?it/s, Train_Loss=0.108, Val_Loss=1.793, Acc=0.723, Epoch=199.0, Fold=004.0]

dense model has accuracy on test set=0.72%
dense model has size=0.09 MiB
The time inference of base model is =0.0059969425201416016
The number of parametrs of base model is:23607


In [17]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)

Eva_final.update({'base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'base_model_size':base_model_size_mean})
Eva_final.update({'Std of base_model_size':base_model_size_std})


print(f"All measurement about Quatization process ")   
Eva_final

All measurement about Quatization process 


{'base model accuracy': 0.712,
 'Std of base model accuracy': 0.017,
 'time inference of base model': 0.009,
 'Std of time inference of base model': 0.007,
 'number parmameters of base model': 23607,
 'Std of number parmameters of base model': 0.0,
 'base_model_size': 755424,
 'Std of base_model_size': 0.0}

In [18]:
# Create a table
table_data = [Base_model_accuracy,T_base_model,Num_parm_base_model, Base_model_size]
            
headers=['1', '2', '3', '4','5']
# Print the table
tabulate(table_data, headers, tablefmt='fancy_grid')
# New column data
first_column_data =  ['Base_model_accuracy','T_base_model','Num_parm_base_model', 'Base_model_size']

# Add a custom index column
headers=['1', '2', '3', '4', '5']
table_with_index = tabulate(table_data, headers=['parameters'] + headers,
                            showindex=first_column_data, tablefmt="fancy_grid", numalign="center")

# Print the extended table
print(table_with_index)

╒═════════════════════╤═══════════╤═══════════╤═══════════╤══════════╤════════════╕
│ parameters          │     1     │     2     │     3     │    4     │     5      │
╞═════════════════════╪═══════════╪═══════════╪═══════════╪══════════╪════════════╡
│ Base_model_accuracy │ 0.707948  │ 0.704251  │ 0.691312  │ 0.735675 │  0.722736  │
├─────────────────────┼───────────┼───────────┼───────────┼──────────┼────────────┤
│ T_base_model        │ 0.0066452 │ 0.0156257 │ 0.0156245 │    0     │ 0.00599694 │
├─────────────────────┼───────────┼───────────┼───────────┼──────────┼────────────┤
│ Num_parm_base_model │   23607   │   23607   │   23607   │  23607   │   23607    │
├─────────────────────┼───────────┼───────────┼───────────┼──────────┼────────────┤
│ Base_model_size     │  755424   │  755424   │  755424   │  755424  │   755424   │
╘═════════════════════╧═══════════╧═══════════╧═══════════╧══════════╧════════════╛
